# Projeto 2 - Classificador Automático de Sentimento

## Integrantes: Lucas Leal Vale, Matheus Augusto Soares, Rafael Almada

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___
## Introdução

Vivemos na era da informação, tudo pode ser usado para o bem ou para o mal, desde o que o indíviduo tá sentido, seja relacionado a uma empresa, produto ou até por alguma questão qualquer, o lugar por onde as pessoas passam, o que curtem em uma rede social, tudo pode ser usado para entender o comortamento social de um nicho específico da sociedade ou até traçar uma média geral para a nação. Com o avanço do conhecimento e do desenvolvimento do poderio da tecnologia, o ser humano percebeu que não precisa depender apenas de si, até por que ele mesmo é muito falho, mas podemos nos utilizar de máquinas não só para registrar, mas para pensar. Assim nasceu o conhecido 'Machine Learning', área da computação que visa programar um computador para aprender com o humano e por meio de probabilidades realizar o mesmo trabalho.

Neste trabalho, utilizamos de uma técnica muito simples de Machine Learning, chamada "Naive Bayes" (traduzido literalmente para Bayes ingênuo). O nome faz uma homenagem a Thomas Bayes (1701 – 1761), matemático inglês que estudou como calcular a distribuição para o parâmetro de probabilidade de uma distribuição binomial e acabou por chegar na famosa equação de probabilidades, a equação de Bayes:

### $$\frac{P(A | B) = P(B | A)·P(A)}{P(B)}$$

Contudo, nosso objetivo é analisar qual a reputação da empresa 'Riot Games', com base no seu público no twitter, para isso recolhemos um dataframe de 300 tweets de treinamento para que o algoritmo desenvolvido possa tomar como base e 700 tweets de teste para verificarmos o quão preciso ele é. A importância de um projeto como esse para uma empresa seria a informação estatística de como ela é vista pelo público em determinada rede social para se embasar em decisões futuras de investimentos, marketing, etc.

___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

In [2]:
df=pd.read_excel('tweets_riot.xlsx','Treinamento',sep=',')
dfTeste=pd.read_excel('teste.xlsx', 'Treinamento',sep=',')

In [3]:
lista1=['  ','   ',':','(',')','`','[',']','.','/','" ',"'"]
lista2=[' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ']
for i in range(len(lista1)):
    df['Treinamento']=df.Treinamento.str.replace(lista1[i], lista2[i])
df['Treinamento']=df.Treinamento.str.lower()

In [4]:
df['Treinamento']=df['Treinamento'].str.split()
df.head()

,Treinamento,Classificação,Legenda
0,"[n, é, pq, eu, tô, com, uma, crush, imensa, nã...",0,0 - IRRELEVANTE
1,"[rt, @favstommo, ai, cansei, de, viver, mas, n...",0,1 - RUIM
2,"[@carlosruaror, consegui,, riot, me, enviou, o...",3,2 - NEUTRO
3,"[riot, fala, assim, mesmo, rs, https, t, co, r...",0,3-BOM
4,"[rt, @saradocinzas, aaaaaaaaaaaaa, vai, toma, ...",1,NaN


___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

In [5]:
dfLen=len(df)

dfBom=df[df['Classificação']==3]
dfBomLen=len(dfBom)

dfNeutro=df[df['Classificação']==2]
dfNeutroLen=len(dfNeutro)

dfRuim=df[df['Classificação']==1]
dfRuimLen=len(dfRuim)

dfIrre=df[df['Classificação']==0]
dfIrreLen=len(dfIrre)
#tw=dfBom["Treinamento"][2]



In [6]:
# # Probabilidades base
# ProbBom=len(dfBom)/(dfLen)
# ProbRuim=len(dfRuim)/(dfLen)
# ProbIrre=len(dfIrre)/(dfLen)
# print('A probablidade de ser Bom  é:{0}'.format(ProbBom))
# print('A probablidade de ser Ruim é:{0}'.format(ProbRuim))
# print('A probablidade de ser Irre é:{0}'.format(ProbIrre))


In [7]:
# Contagem de palavras dado sua categoria
#Bom
dfBomProb=dfBom["Treinamento"].sum()
dfBomProb=pd.Series(dfBomProb)
dfBomProb=dfBomProb.value_counts()

#Ruim
dfRuimProb=dfRuim["Treinamento"].sum()
dfRuimProb=pd.Series(dfRuimProb)
dfRuimProb=dfRuimProb.value_counts()

#Irrelevante
dfIrreProb=dfIrre["Treinamento"].sum()
dfIrreProb=pd.Series(dfIrreProb)
dfIrreProb=dfIrreProb.value_counts()

#Neutro
dfNeutroProb=dfNeutro["Treinamento"].sum()
dfNeutroProb=pd.Series(dfNeutroProb)
dfNeutroProb=dfNeutroProb.value_counts()

#Total
dfTotal=df.Treinamento.sum()
dfTotal=pd.Series(dfTotal)
dfTotal=dfTotal.value_counts()

In [8]:
#Número total de palavras no dataframe
dfPalavras=df.Treinamento.sum()
dfPalavras=pd.Series(dfPalavras)
dfPalavras=len(dfPalavras)

#Check de palavras
check=df.Treinamento.sum()
check=pd.Series(check)

dfPalavras

5523

In [9]:
#Números totais de palavras em cada categoria, mas agora em variáveis
#Bom
BoasPalavras=dfBom["Treinamento"].sum()
BoasPalavras=pd.Series(BoasPalavras)
BoasPalavras=len(BoasPalavras)

#Ruim
RuimPalavras=dfRuim["Treinamento"].sum()
RuimPalavras=pd.Series(RuimPalavras)
RuimPalavras=len(RuimPalavras)

#Irrelevante
IrrePalavras=dfIrre["Treinamento"].sum()
IrrePalavras=pd.Series(IrrePalavras)
IrrePalavras=len(IrrePalavras)

#Neutro
NeutroPalavras=dfNeutro["Treinamento"].sum()
NeutroPalavras=pd.Series(NeutroPalavras)
NeutroPalavras=len(NeutroPalavras)


In [10]:
# Probabilidades base
ProbBom=BoasPalavras/(dfPalavras)
ProbRuim=RuimPalavras/(dfPalavras)
ProbIrre=IrrePalavras/(dfPalavras)
ProbNeutro=NeutroPalavras/(dfPalavras)
print('A probablidade de ser Bom  é: {0:.3f} %'.format(ProbBom*100))
print('A probablidade de ser Ruim é: {0:.3f} %'.format(ProbRuim*100))
print('A probablidade de ser Irrelevante é: {0:.3f} %'.format(ProbIrre*100))
print('A probablidade de ser Neutro é: {0:.3f} %'.format(ProbNeutro*100))

A probablidade de ser Bom  é: 8.564 %
A probablidade de ser Ruim é: 31.505 %
A probablidade de ser Irrelevante é: 34.619 %
A probablidade de ser Neutro é: 25.312 %


In [11]:
#P(palavra|Bom)
#Prob palavra/bom
#dfBomProb2=(dfBomProb+1)/(len(dfBomProb)+len(dfTotal))
#Prob palavra/ruim
#dfRuimProb2=(dfRuimProb+1)/(len(dfRuimProb)+len(dfTotal))
#Prob palavra/irrelevante
#dfIrreProb2=(dfIrreProb+1)/(len(dfIrreProb)+len(dfTotal))
#dfBomProb2.head(5)

In [12]:
def calcula_condicional(df,den):
    dfBomProb2=(dfBomProb+1)/(BoasPalavras+den)
    dfRuimProb2=(dfRuimProb+1)/(RuimPalavras+den)
    dfIrreProb2=(dfIrreProb+1)/(IrrePalavras+den)
    dfNeutroProb2=(dfNeutroProb+1)/(NeutroPalavras+den)
    return dfBomProb2, dfRuimProb2, dfIrreProb2, dfNeutroProb2

In [13]:
# dfBomProb2=(dfBomProb+1)/(BoasPalavras+dfPalavras)
# dfRuimProb2=(dfRuimProb+1)/(RuimPalavras+dfPalavras)
# dfIrreProb2=(dfIrreProb+1)/(IrrePalavras+dfPalavras)
# dfNeutroProb2=(dfNeutroProb+1)/(NeutroPalavras+dfPalavras)

In [14]:
# for i in df.Treinamento:
#     for u in i:
#         print("u: ", u)
#         print("dft u:", dfTotal[u])


In [15]:
resultado=[]

In [16]:
def bayes(df,dfPalavras,lista):
    den=0
    for j in df:
        for k in j:
            if k not in check:
                den+=1
    x=calcula_condicional(df, den)
    dfBomProb2=x[0]
    dfRuimProb2=x[1]
    dfIrreProb2=x[2]
    dfNeutroProb2=x[3]
    laplace=(1/(den))# obs da folha
    for i in df: # for i in df.index:
        c_bom=1
        c_ruim=1
        c_irre=1
        c_neutro=1
        for u in i:
            #p=dfTotal[u]/len(dfTotal) #(Probabilidade da palavra no espaço amostral de todas as palavras)
            if u not in dfBomProb2:
                p_bom=laplace
            else:
                p_bom=dfBomProb2[u]
            if u not in dfRuimProb2:
                p_ruim=laplace
            else:
                p_ruim=dfRuimProb2[u]
            if u not in dfIrreProb2:
                p_irre=laplace
            else:
                p_irre=dfIrreProb2[u]
            if u not in dfNeutroProb2:
                p_neutro=laplace
            else:
                p_neutro=dfNeutroProb2[u]
                
            c_bom*=(p_bom)
            c_ruim*=(p_ruim)
            c_irre*=(p_irre)
            c_neutro*=(p_neutro)
        c_bom=c_bom*(ProbBom)
        #print("c_bom:",c_bom)
        c_ruim=c_ruim*(ProbRuim)
        #print("c_ruim:",c_ruim)
        c_irre=c_irre*(ProbIrre)
        #print("c_irre:",c_irre)
        c_neutro=c_neutro*(ProbNeutro)

        if c_bom>c_ruim and c_bom>c_irre and c_bom>c_neutro:
            #Classifica como Bom (3)
            lista.append(3)

        elif c_ruim>c_bom and c_ruim>c_irre and c_ruim>c_neutro:
            #Classifica como Ruim (1)
            lista.append(1)
        elif c_irre>c_bom and c_irre>c_ruim and c_irre>c_neutro:
            #Classifica como irrelevante (0)
            lista.append(0)
        else:
            #Classifica como neutro (2) 
            lista.append(2)
    return lista

In [17]:
resultado=[]
a=bayes(df["Treinamento"],dfPalavras,resultado)

In [18]:
print(len(resultado))
print(a)

300
[0, 0, 2, 2, 1, 0, 0, 1, 0, 2, 1, 2, 1, 1, 2, 0, 1, 0, 2, 2, 1, 0, 0, 2, 0, 0, 2, 1, 0, 2, 1, 1, 0, 1, 2, 0, 0, 0, 2, 1, 0, 2, 0, 1, 2, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 3, 1, 2, 1, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 2, 2, 0, 0, 0, 0, 1, 0, 1, 0, 2, 0, 0, 1, 0, 0, 0, 1, 0, 2, 2, 1, 0, 1, 0, 0, 0, 0, 0, 2, 2, 1, 2, 0, 2, 1, 1, 2, 2, 1, 1, 1, 0, 1, 2, 0, 0, 2, 0, 1, 1, 1, 0, 1, 1, 3, 2, 0, 2, 1, 0, 1, 0, 0, 2, 2, 2, 0, 1, 1, 2, 0, 0, 0, 2, 1, 0, 0, 1, 1, 1, 2, 0, 1, 2, 2, 2, 0, 1, 2, 1, 1, 1, 0, 0, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 0, 2, 1, 2, 0, 1, 0, 1, 3, 2, 0, 2, 0, 3, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 2, 1, 2, 2, 0, 2, 0, 1, 1, 2, 0, 1, 2, 0, 1, 1, 0, 0, 2, 2, 2, 2, 2, 0, 2, 0, 0, 0, 1, 0, 0, 2, 1, 1, 3, 0, 2, 1, 2, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 2, 0, 0, 1, 0, 2, 1, 0, 1, 0, 2, 1, 2, 2, 1, 2, 1, 2, 2, 2, 1, 1, 0, 1, 2, 2, 1, 1, 2, 0, 2, 3, 0, 1, 1, 2, 1, 2, 2, 0, 1, 3, 2, 2, 2, 1, 0, 1]


In [19]:
df["Resultado"]=a

In [20]:
df.head()

,Treinamento,Classificação,Legenda,Resultado
0,"[n, é, pq, eu, tô, com, uma, crush, imensa, nã...",0,0 - IRRELEVANTE,0
1,"[rt, @favstommo, ai, cansei, de, viver, mas, n...",0,1 - RUIM,0
2,"[@carlosruaror, consegui,, riot, me, enviou, o...",3,2 - NEUTRO,2
3,"[riot, fala, assim, mesmo, rs, https, t, co, r...",0,3-BOM,2
4,"[rt, @saradocinzas, aaaaaaaaaaaaa, vai, toma, ...",1,NaN,1


In [21]:
pd.crosstab(df['Classificação'], df['Resultado'])

Resultado,0,1,2,3
Classificação,,,,
0,108,1,8,0
1,0,80,3,0
2,0,8,67,0
3,2,10,6,7


___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [22]:
r=[]
b=bayes(dfTeste['Teste'], dfPalavras,r)
dfTeste['Resultado']=b

In [23]:
dfTeste

,Teste,Classificação,Legenda,Resultado
0,gostei de um vídeo @youtube https://t.co/4oilk...,2.0,0 - IRRELEVANTE,2
1,"rt @lolesportsbr: ""senhor do meu destino mano,...",3.0,1 - RUIM,2
2,"rt @ahsbrasil: sinopse ep. 01 - ""the end""\n\nn...",0.0,2 - NEUTRO,2
3,"@yetz1 riot ta de sacanagem só pode, nunca viu...",1.0,3 - BOM,2
4,"@guilhermdw vc não tá entendendo, não é uma va...",3.0,NaN,2
5,riot cade meu rp,1.0,NaN,2
6,@giogrosso_dsr é o mid season invitational. ac...,2.0,NaN,2
7,riot muito sensata https://t.co/aixjlyvpyg,2.0,NaN,2
8,"rt @yetz1: riot pensa em tanta coisa, pq não p...",1.0,NaN,2
9,riot eu to tentando parar de jogar pfvr colabo...,2.0,NaN,2


In [24]:
pd.crosstab(dfTeste['Classificação'],dfTeste['Resultado'])

Resultado,0,1,2
Classificação,,,
0.0,1,11,123
1.0,3,11,169
2.0,6,3,49
3.0,5,23,292


___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
